# RADICAL-EnTK with Darshan to generate workflow provenance graph

## Darshan

https://wordpress.cels.anl.gov/darshan/

Darshan is an HPC I/O characterization tool. It is designed to capture an accurate picture of application I/O behavior, including properties such as patterns of access within files, with minimum overhead. Darshan can be used to investigate and tune the I/O behavior of complex HPC applications.

In [ ]:
%%capture
# install darshan or ensure that it is installed
!./scripts/darshan-install.sh

## Example of enabling Darshan

In the example, Darshan is enabled for a specific `re.Task` and for all tasks within a specific `re.Stage`.

In [ ]:
import os

import radical.entk  as re
import radical.pilot as rp

from radical.entk.tools import (cache_darshan_env,
                                with_darshan,
                                enable_darshan,
                                get_provenance_graph)

In [ ]:
%env RADICAL_REPORT_ANIME=FALSE
%env RADICAL_REPORT=TRUE
%env RADICAL_LOG_LVL=DEBUG

In [ ]:
%%capture
# prepare darshan environment
#    runtime root dir  (if not provided then env variable $DARSHAN_RUNTIME_ROOT should be set)
#    modules           (optional)
#    env variables     (optional)
cache_darshan_env(darshan_runtime_root='/usr/local',
                  modules=[], env={})

In [ ]:
TASK_01_OUTPUT = 'output_01.dat'


def get_stage_0():

    # hello-RP task
    task_00 = re.Task({
        'executable': 'radical-pilot-hello.sh',
        'arguments' : [10],
        'cpu_reqs'  : {'cpu_processes'  : 1,
                       'cpu_threads'    : 4,
                       'cpu_thread_type': rp.OpenMP}
    })

    # R/W data
    task_01   = re.Task({
        'executable'       : '/bin/sh',
        'arguments'        : ['-c', f'cat input.dat | wc > {TASK_01_OUTPUT}'],
        'upload_input_data': ['/etc/passwd > input.dat'],
        'copy_output_data' : [f'{TASK_01_OUTPUT} > $SHARED/{TASK_01_OUTPUT}']
    })

    stage_0 = re.Stage()
    # --- enable Darshan for task "task_01" only
    stage_0.add_tasks([task_00, enable_darshan(task_01)])
    return stage_0


# --- enable Darshan for the whole "stage_1" using decorator
@with_darshan
def get_stage_1():

    # R/W data and task depends on the task from the previous stage
    task_10 = re.Task({
        'executable'     : '/bin/sh',
        'arguments'      : ['-c',
                            f"sed -r 's/\s+//g' {TASK_01_OUTPUT} " +  # noqa: W605
                            '| grep -o . | sort | uniq -c > output_10.dat'],
        'copy_input_data': [f'$SHARED/{TASK_01_OUTPUT} > {TASK_01_OUTPUT}']
    })

    stage_1 = re.Stage()
    stage_1.add_tasks([task_10])
    return stage_1

In [ ]:
amgr = re.AppManager()
amgr.resource_desc = {'resource': 'local.localhost',
                      'cpus'    : 8,
                      'walltime': 15}

pipeline = re.Pipeline()
pipeline.add_stages([get_stage_0(), get_stage_1()])

amgr.workflow = [pipeline]
amgr.run()

In [ ]:
sid = amgr.sid

!ls -al ~/radical.pilot.sandbox/"$sid"/pilot.0000/task.000001/darshan_logs

In [ ]:
from pprint import pformat

print(pformat(get_provenance_graph(pipelines=[pipeline],
                                   output_file='entk_provenance.json')))